# Main notebook

In [54]:
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy
import pandas as pd
import scipy.stats as stats
import scipy.sparse as sparse
import copy
import time
import warnings 
from scipy import integrate
import seaborn as sns
from numpy import sqrt, cosh, tanh, pi, exp
warnings.simplefilter('ignore')
from sklearn.decomposition import PCA, FactorAnalysis

import scipy.io as sio
import os
from tqdm.auto import tqdm

# using functions from PyalData
import pyaldata as pld

# matlab2python functions
import tools 

In [14]:
percent_spikes = [1,2,3,5,10]
num_iters = 10
which_dim = 1
binz = np.arange(0, 0.005, 0.5) # start, step, stop

## 0  Load data and convert to df

In [22]:
'''
Insert path and filename 
'''
path = '/Users/Daphne/Data/'  
fname = 'Chewie_CO_VR_2016-10-06.mat' 
mat = scipy.io.loadmat(path + fname) 

df = pld.mat2dataframe(path+fname, shift_idx_fields=True)

In [23]:
df.shape # all the data

(774, 24)

In [24]:
df.head()

,monkey,date,task,target_direction,trial_id,result,bin_size,perturbation,perturbation_info,epoch,...,idx_peak_speed,idx_trial_end,pos,vel,acc,force,M1_spikes,M1_unit_guide,PMd_spikes,PMd_unit_guide
0,Chewie,10-06-2016,CO,-2.356194,1,R,0.01,VR,-0.52,BL,...,363,436,"[[-5.399950889827338, -25.11058978269873], [-5...","[[4.615264447089089, -3.6781866137904946], [4....","[[34.913180411358354, -17.92603186704403], [13...","[[0.3854857746577438, -0.18448026169727955], [...","[[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,...","[[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2..."
1,Chewie,10-06-2016,CO,1.570796,2,R,0.01,VR,-0.52,BL,...,356,447,"[[-5.194667657852522, -35.190944353130526], [-...","[[2.3921738136280783, 2.833707639241631], [2.7...","[[31.99226756476989, 80.42711312188004], [34.1...","[[0.1778157378731141, 0.12542049016429982], [0...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,...","[[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2..."
2,Chewie,10-06-2016,CO,0.785398,3,R,0.01,VR,-0.52,BL,...,414,483,"[[0.35267826312911055, -22.923278550949004], [...","[[0.5017102009485903, -3.146994710197926], [0....","[[-7.180978950112355, -35.21789901845524], [3....","[[0.2833417683274762, -0.1285513884878812], [0...","[[0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...","[[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...","[[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2..."
3,Chewie,10-06-2016,CO,3.141593,4,R,0.01,VR,-0.52,BL,...,397,458,"[[5.072834875801728, -24.89410841851521], [5.0...","[[-1.3275097541588876, -3.5308863978243985], [...","[[-6.932033881542791, 0.17982626642873387], [-...","[[0.08675043530985374, 0.03658922221440793], [...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2..."
4,Chewie,10-06-2016,CO,-1.570796,5,R,0.01,VR,-0.52,BL,...,301,362,"[[-8.194254800472459, -30.26359832666607], [-8...","[[1.8501123719868446, 0.8394848146862552], [2....","[[48.123169514922175, -9.081942493745938], [58...","[[0.2054977133197953, -0.12485484173352068], [...","[[0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,...","[[5, 1], [5, 2], [5, 3], [5, 4], [9, 1], [9, 2...","[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[1, 1], [1, 2], [2, 1], [2, 2], [3, 1], [3, 2..."


In [31]:
df.columns

Index(['monkey', 'date', 'task', 'target_direction', 'trial_id', 'result',
       'bin_size', 'perturbation', 'perturbation_info', 'epoch',
       'idx_trial_start', 'idx_target_on', 'idx_go_cue', 'idx_movement_on',
       'idx_peak_speed', 'idx_trial_end', 'pos', 'vel', 'acc', 'force',
       'M1_spikes', 'M1_unit_guide', 'PMd_spikes', 'PMd_unit_guide'],
      dtype='object')

## 1 Preprocess 

### Combine time bins

In [26]:
td = pld.combine_time_bins(df, 2)

In [28]:
print(td.head().bin_size)

0    0.02
1    0.02
2    0.02
3    0.02
4    0.02
Name: bin_size, dtype: float64


### Remove bad neurons (Note, this ONLY removes the lower fr neurons. Add functionality?)

In [47]:
td = pld.remove_low_firing_neurons(td, 'M1_spikes',  5)
td = pld.remove_low_firing_neurons(td, 'PMd_spikes', 5)

print(f'from {df.M1_spikes[0].shape} neurons to {td.M1_spikes[0].shape} ')

from (447, 63) neurons to (223, 44) 


### Transform signals

In [53]:
td = pld.transform_signal(td, 'M1_spikes',  'sqrt')
td = pld.transform_signal(td, 'PMd_spikes', 'sqrt')

td.M1_spikes

0      [[1.0, 1.0, 0.0, 1.189207115002721, 0.0, 0.0, ...
1      [[1.0, 1.0, 0.0, 0.0, 0.0, 1.189207115002721, ...
2      [[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0,...
3      [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4      [[0.0, 0.0, 1.0, 1.189207115002721, 1.0, 1.0, ...
                             ...                        
769    [[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
770    [[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
771    [[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.18...
772    [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...
773    [[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,...
Name: M1_spikes, Length: 774, dtype: object

## 2 Dimensionality reduction

In [56]:
dim = 10

#pld.dim_reduce(td, PCA(dim), 'both_rates', 'both_pca')

In [49]:
# '''
# Session parameters
# '''

# sessions = [...] # TODO: for all sessions (currently only doing one session)

# task = 'CO'

# # set the start and end indices 
# num_bins = 3
# which_period = 'all' 

# if which_period == 'plan':
#     idx_start = df['idx_target_on'][0]
#     idx_end = df['idx_go_cue'][0]
    
# elif which_period == 'move':
#     idx_start = df['idx_go_cue'][0]
#     idx_end = df['idx_trial_end'][0]

# elif which_period == 'all':
#     idx_start = df['idx_target_on'][0]
#     idx_end = df['idx_trial_end'][0]

In [50]:
#eigen_sum = np.zeros([len(sessions), 3]) #TODO: where does the 3 come from??